## Imports

In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression, LogisticRegression, SGDClassifier, BayesianRidge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm

from sklearn.metrics import mean_squared_error, r2_score
from datetime import datetime

DATEPARSE = "%Y-%m-%d %H:%M:%S"

# Read data
### Add data into /taxi_data/
### Github stores a sample file of this. Download the rest and do the same.

In [27]:
def add_zero_padding(number):
    if len(str(number)) == 1: return "0" + str(number)
    else: return number
def format_df(DF, pickup_date_column, dropof_date_column):
    for i, x in DF.iterrows():
        DF.at[i,"month"] = 1
        start = datetime.strptime(x[pickup_date_column], DATEPARSE).time()
        stop = datetime.strptime(x[dropof_date_column], DATEPARSE).time()

        DF.at[i, pickup_date_column] = (start.hour * 60 + start.minute)
        DF.at[i, dropof_date_column] = (stop.hour * 60 + stop.minute)
        DF.at[i, "travel_time"] = (start.hour * 60 + start.minute) - (stop.hour * 60 + stop.minute)
    return DF

In [28]:
#GREEN
header = True
for x in range(1, 13): # this takes 14 min
    try:
        dfGreen = pd.read_csv("taxi_data\green_tripdata_2019-" + str(add_zero_padding(x)) + ".csv", usecols=["PULocationID", "DOLocationID", "lpep_pickup_datetime", "lpep_dropoff_datetime"])
        dfGreen = format_df(dfGreen, "lpep_pickup_datetime", "lpep_dropoff_datetime")
        dfGreen.to_csv("taxi_data\green_tripdata_2019_concat.csv", mode='a+', header=header, index=False)
        header=False
        print("GREEN: " + str(x))
    except Exception as f:
        print(f)

"""
# YELLOW
header = True
for x in range(1, 13):
    try:
        dfYellow = pd.read_csv("taxi_data\yellow_tripdata_2019-" + str(add_zero_padding(x)) + ".csv")
        dfYellow = format_df(dfYellow, "tpep_pickup_datetime", "tpep_dropoff_datetime")
        dfYellow.to_csv("taxi_data\yellow_tripdata_2019_concat.csv", mode='a+', header=header, index=False)
        header=False
        print("YELLOW: " + str(x))
    except Exception as f:
        print(f)
"""

unsupported operand type(s) for -: 'datetime.time' and 'datetime.time'
unsupported operand type(s) for -: 'datetime.time' and 'datetime.time'
[Errno 2] No such file or directory: 'taxi_data\\green_tripdata_2019-03.csv'
unsupported operand type(s) for -: 'datetime.time' and 'datetime.time'


KeyboardInterrupt: 

# Train

In [11]:
dfGreen = pd.read_csv("taxi_data\green_tripdata_2019_concat.csv", chunksize=100000)

model = KNeighborsClassifier(3)

first = True
for chunk in dfGreen:
    print(chunk)
    X_green = chunk[["PULocationID", "DOLocationID","lpep_pickup_datetime", "month"]] # input cols
    y_green = chunk[["travel_time"]] # output cols
    print(X_green[0])
    

    X_train, X_test, y_train, y_test = train_test_split(X_green, y_green, test_size = 0.25)

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    print("Mean squared error: %.2f" % mean_squared_error(y_test, y_pred))
    print("Coefficient of determination: %.2f" % r2_score(y_test, y_pred))
    
    break

       lpep_pickup_datetime  lpep_dropoff_datetime  PULocationID  \
0              3.754394e+09           3.754394e+09           264   
1              3.755290e+09           3.755291e+09            97   
2              3.755291e+09           3.755291e+09            49   
3              3.755292e+09           3.755293e+09           189   
4              3.755291e+09           3.755292e+09            82   
...                     ...                    ...           ...   
99995          3.755757e+09           3.755758e+09           130   
99996          3.755755e+09           3.755757e+09           218   
99997          3.755754e+09           3.755756e+09           139   
99998          3.755757e+09           3.755758e+09            72   
99999          3.755758e+09           3.755759e+09            89   

       DOLocationID  month  travel_time  
0               264    1.0         88.0  
1                49    1.0        376.0  
2               189    1.0        267.0  
3              

KeyError: 0